In [10]:
# We first install the packages
%pip install crewai langchain langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [11]:
# We install our environment variables helper library
%pip install python-dotenv --quiet
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [12]:
# We load our helper callback to see what goes over the wire
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

In [13]:
# We load the llm mode using our helper and based onthe settings
import _models
import _settings
llm=_models.get_llm(_settings.MODEL_NAME, callbacks=[callback])


In [14]:
# We load our challenge from the settings
import _settings
challenge_prompt=_settings.CHALLENGE_PROMPT

In [15]:
from crewai import Agent, Task, Crew, Process
import _agents
developer= _agents.get_agent_by_name("senior developer",llm)
ops = _agents.get_agent_by_name("senior devops",llm)

Creating senior developer
Creating senior devops


In [16]:
# Create tasks for your agents
task1 = Task(
  description=challenge_prompt,
  expected_output="A script shown as markdown code block",
  agent=developer,
  llm=llm
)

In [17]:
# Create tasks for your agents
task2 = Task(
  description="""Analyze a shell script and make sure it's compatible with most of the Linux systems.""",
  expected_output="A script shown as markdown code block",
  agent=ops,
  llm=llm
)

In [18]:

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[developer],
  tasks=[task1,task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
  full_output=True,
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-06-15 15:03:46,810 - 8387709952 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Developer
 [INFO]: == Starting Task: Write me a shell script to list the files in the current directory. 
Output the script as markdown code block


> Entering new CrewAgentExecutor chain...
I can write a simple shell script that lists all the files in the current directory. I will use the `ls` command, which is a standard command to list files in Unix and Unix-like operating systems. I will output the script as a markdown code block as requested.

Final Answer:

Here is the shell script as a markdown code block:

```bash
#!/bin/bash
# This script will list all the files in the current directory

ls
```

To run this script, save it to a file, give it execute permissions using `chmod +x filename`, and then run it with `./filename`. It will list all the files in the directory where it is run.

> Finished chain.
 [DEBUG]: == [Senior Developer] Task output: Here is the shell script as a markdown code block:

```bash
#!/bin/bash
# This script will list all